In [1]:
# Install necessary libraries
!pip3 install transformers torch ipywidgets

## GPT 2: Foundation Model

In [2]:
# Import necessary libraries
import ipywidgets as widgets
import torch

from transformers import pipeline, set_seed
from IPython.display import display

In [3]:
# Check if GPU is available, else fallback to MPS or CPU
# device = "gpu" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = "cpu"
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Load the text generation pipeline
generator = pipeline('text-generation', model='gpt2', device=device)

set_seed(42)

In [5]:
# Create input text box and output display area
text_input = widgets.Text(
    value='',
    placeholder='Enter text here',
    description='Input:',
    disabled=False,
    layout=widgets.Layout(overflow_x = 'auto', display= 'inline-flex', flex_flow= 'row wrap')
)

# Style the button to be green
generate_button = widgets.Button(
    description="Generate Text",
    button_style='', # Setting to '' allows custom CSS
    layout=widgets.Layout(width="20%")
)
generate_button.style.button_color = 'lightgreen'  # This sets a light green color to the button

text_output = widgets.Output(layout=widgets.Layout(overflow_x = 'auto', display= 'inline-flex', flex_flow= 'row wrap'))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [6]:
# Function to handle text generation when the button is clicked
def text_button_click(b):
    prompt = text_input.value
    
    # Encode the input and move it to the selected device
    inputs = generator.tokenizer(prompt, return_tensors="pt")
    inputs['input_ids'] = inputs['input_ids'].to(device)
    inputs['attention_mask'] = inputs['attention_mask'].to(device)

    # Generate text with attention_mask and pad_token_id
    generated_texts = generator.model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=True,
        max_length=256,
        num_beams=3,
        repetition_penalty=2.0,
        early_stopping=True,
        num_return_sequences=1, 
        pad_token_id=generator.tokenizer.pad_token_id
    )
    
    # Decode the generated texts
    decoded_texts = [generator.tokenizer.decode(gen, skip_special_tokens=True) for gen in generated_texts]
    
    # Clear previous output and display new results
    with text_output:
        text_output.clear_output()
        for i, gen_text in enumerate(decoded_texts):
            print(f"Generated Text {i+1}: {gen_text}\n")

In [7]:
# Attach the button click event
generate_button.on_click(text_button_click)

# Display the widgets
display(text_input, generate_button, text_output)

Text(value='', description='Input:', layout=Layout(display='inline-flex', flex_flow='row wrap'), placeholder='…

Button(description='Generate Text', layout=Layout(width='20%'), style=ButtonStyle(button_color='lightgreen'))

Output(layout=Layout(display='inline-flex', flex_flow='row wrap'))

## Astro GPT: A fine-tuned version of GPT-2 model

In [8]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("stevhliu/astroGPT")
model = AutoModelForCausalLM.from_pretrained("stevhliu/astroGPT").to(device)

In [10]:
# Create input text box and output display area
zodiac_input = widgets.Text(
    value='',
    placeholder='Enter date here (exp, Sep 02, 2020)',
    description='Input:',
    disabled=False,
    layout=widgets.Layout(overflow_x = 'auto', display= 'inline-flex', flex_flow= 'row wrap')
)

# Style the button to be green
horoscope_button = widgets.Button(
    description="Generate Horoscope",
    button_style='', # Setting to '' allows custom CSS
    layout=widgets.Layout(width="20%")
)
horoscope_button.style.button_color = 'lightgreen'  # This sets a light green color to the button

zodiac_output = widgets.Output(layout=widgets.Layout(overflow_x = 'auto', display= 'inline-flex', flex_flow= 'row wrap'))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
# Function to handle text generation when the button is clicked
def zodiac_button_click(b):
    prompt = zodiac_input.value
    
    # Encode the input and move it to the selected device
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate text with attention_mask and pad_token_id
    sample_output = model.generate(
        input_ids,
        do_sample=True,
        max_length=256,
        num_beams=3,
        repetition_penalty=2.0,
        early_stopping=True,
        num_return_sequences=1, 
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode the output tokens to get the text
    generated_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    
    # Clear previous output and display new results
    with zodiac_output:
        zodiac_output.clear_output()
        print(f"Generated Text: {generated_text}\n")

In [12]:
# Attach the button click event
horoscope_button.on_click(zodiac_button_click)

# Display the widgets
display(zodiac_input, horoscope_button, zodiac_output)

Text(value='', description='Input:', layout=Layout(display='inline-flex', flex_flow='row wrap'), placeholder='…

Button(description='Generate Horoscope', layout=Layout(width='20%'), style=ButtonStyle(button_color='lightgree…

Output(layout=Layout(display='inline-flex', flex_flow='row wrap'))